In [1]:
import pandas as PD
import numpy as NP
import requests
from bs4 import BeautifulSoup
from selenium import webdriver  
from selenium.common.exceptions import TimeoutException  
# import ActionChains mouse operation  
from selenium.webdriver.common.action_chains import ActionChains 

from selenium.webdriver.chrome.options import Options
import time
from lxml import etree


# Dynamic get html

In [12]:
def get_html(url):
    try:
        chrome_options=Options()
        chrome_options.add_argument('--headless')
        driver= webdriver.Chrome(chrome_options=chrome_options)  
        driver.get(url)
        temp_height=0
 
        while True:
            #循环将滚动条下拉
            driver.execute_script("window.scrollBy(0,100000)")
            #sleep一下让滚动条反应一下
            time.sleep(5)
            #获取当前滚动条距离顶部的距离
            check_height = driver.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
            #如果两者相等说明到底了
            if check_height==temp_height:
                break
            temp_height=check_height
            print('*****Processing temp_height = {temp_height}*****'.format(temp_height=temp_height))
        print('done')
        page = driver.page_source
        source = page.encode("utf8")
        soup_list = BeautifulSoup(source, 'lxml').body.find('div', id="root-react-container").find("div", class_ = "boards-body fenbushi-capital").find("div", class_ = "job-list")
        return soup_list
    except:
        return "ERROR"

In [13]:
def get_green_house_url(url):
    try:
        response  = requests.get(url)
        response.encoding="utf-8"
        source = response.text
        content = BeautifulSoup(source, 'lxml').body.find('div', id="content").get_text()
        return content
    except:
        return "ERROR"

In [14]:
# description = get_green_house_url('https://boards.greenhouse.io/mythicalgames/jobs/5463953003')
# description

In [15]:
soup_list = get_html("https://careers.fenbushicapital.vc/jobs")

/tmp/ipykernel_3487/375155834.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 4476*****
*****Processing temp_height = 7562*****
*****Processing temp_height = 11016*****
*****Processing temp_height = 14165*****
*****Processing temp_height = 17303*****
*****Processing temp_height = 20407*****
*****Processing temp_height = 23608*****
*****Processing temp_height = 26761*****
*****Processing temp_height = 29963*****
*****Processing temp_height = 33164*****
*****Processing temp_height = 36383*****
*****Processing temp_height = 39613*****
*****Processing temp_height = 42892*****
*****Processing temp_height = 46034*****
*****Processing temp_height = 49270*****
*****Processing temp_height = 52440*****
*****Processing temp_height = 55701*****
*****Processing temp_height = 58952*****
*****Processing temp_height = 62154*****
*****Processing temp_height = 65439*****
*****Processing temp_height = 68532*****
*****Processing temp_height = 71786*****
*****Processing temp_height = 74942*****
*****Processing temp_height = 78085*****
*****Processing te

In [16]:
# soup_list

In [17]:
# get the initializd infomation
tag_list = []

job_title_list = []
type_list = {}
industry_list = {}
contract_list = {}
location_list = {}
equity_list = {}
description_list = {}
salary_list = {}
label_list = {}
achievements_list = {}
# loop = 0

for tag in soup_list.find_all('div',class_="job-list-job"):
        # print(tag)
        raw_data = tag.find('a', target = "_blank")
        job_title_list.append(raw_data.text)
        type_list[raw_data.text] = 'Full-time'
        industry_list[raw_data.text] = ['Others']
        contract_list[raw_data.text] = 'talentre'
        # location_list[tag.find('a', target = "_blank").text] = tag.find('span', class_ = "job-list-badge-text").text if (not tag.find('span', class_ = "job-list-badge-text").text[0:6] == "Posted") else "Not Posted"
        location_list[raw_data.text] = tag.find('span', class_ = "job-list-company-meta-item").text
        equity_list[raw_data.text] = "Not Posted"
        description_list[raw_data.text] = raw_data['href']
        salary_list[raw_data.text] = "Not Posted"
        label_list[raw_data.text] = []
        achievements_list[raw_data.text] = 'Null'
        for tag_ in tag.find_all('span', class_ = 'job-list-job-skill job-list-job-skill-clickable'):
                label_list[raw_data.text].append(tag_.text)
        # loop += 1
        # if(loop == 10):
        #     break
        # for tag_ in tag.find_all('span',attrs={ 'data-toggle': 'tooltip'}):
        #     if('$' in tag_.text):
        #         tmp = tag_.text.split('$')[1].replace(',','').replace(')','')

In [26]:
#adjust the description_list
for single in job_title_list:
    if ('https://boards.greenhouse.io/' in description_list[single]):
        description_list[single] = get_green_house_url(description_list[single])
        # print('YES')

In [19]:
# adjust the type_list
soup_list_internship = get_html("https://careers.fenbushicapital.vc/jobs?internshipOnly=true")
for tag in soup_list_internship.find_all('div',class_="job-list-job"):
    type_list[tag.find('a', target = "_blank").text] = 'Internship'

/tmp/ipykernel_3487/375155834.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 4555*****
*****Processing temp_height = 7114*****
done


In [22]:
# adjust the type_list
url_list = {f"https://careers.fenbushicapital.vc/jobs?jobTypes=Engineer":"Engineering",
            f"https://careers.fenbushicapital.vc/jobs?markets=FinTech": "Finance",
            f"https://careers.fenbushicapital.vc/jobs?markets=Financial+Services": "Finance",
            f"https://careers.fenbushicapital.vc/jobs?jobTypes=Marketing": "Marketing",
            f"https://careers.fenbushicapital.vc/jobs?markets=Investment%2FTrading": "Non-Tech",
            f"https://careers.fenbushicapital.vc/jobs?markets=Custody": "Customer Support",
            f"https://careers.fenbushicapital.vc/jobs?markets=Infrastructure": "Product",
            f"https://careers.fenbushicapital.vc/jobs?skills=User+Experience+Design": "Design",
            f"https://careers.fenbushicapital.vc/jobs?markets=Stablecoin": "Operations",
            }

In [23]:
for url in url_list.keys():
    print("********************Processing {url} for label {label}********************".format(url = url, label = url_list[url]))
    soup_list_type = get_html(url)
    for tag in soup_list_type.find_all('div',class_="job-list-job"):
        raw_data = tag.find('a', target = "_blank")
        if(raw_data.text in industry_list.keys()):
            if (industry_list[raw_data.text] == ['Others']):
                industry_list[raw_data.text] = [url_list[url]]
            elif (not url_list[url] in industry_list[raw_data.text]):
                industry_list[raw_data.text].append(url_list[url])
            else: 
                continue
        else:
            continue


********************Processing https://careers.fenbushicapital.vc/jobs?jobTypes=Engineer for label Engineering********************


/tmp/ipykernel_3487/375155834.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 4611*****
*****Processing temp_height = 7897*****
*****Processing temp_height = 11098*****
*****Processing temp_height = 14317*****
*****Processing temp_height = 17641*****
*****Processing temp_height = 20836*****
*****Processing temp_height = 24114*****
*****Processing temp_height = 27379*****
*****Processing temp_height = 30602*****
*****Processing temp_height = 33821*****
*****Processing temp_height = 37022*****
*****Processing temp_height = 40227*****
*****Processing temp_height = 43408*****
*****Processing temp_height = 46554*****
*****Processing temp_height = 49822*****
*****Processing temp_height = 52848*****
*****Processing temp_height = 56081*****
*****Processing temp_height = 59164*****
*****Processing temp_height = 62243*****
*****Processing temp_height = 65424*****
*****Processing temp_height = 68643*****
*****Processing temp_height = 71802*****
*****Processing temp_height = 74930*****
*****Processing temp_height = 78223*****
*****Processing te

/tmp/ipykernel_3487/375155834.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 1289*****
done
********************Processing https://careers.fenbushicapital.vc/jobs?markets=Financial+Services for label Finance********************


...

*****Processing temp_height = 4548*****
*****Processing temp_height = 7582*****
*****Processing temp_height = 10622*****
*****Processing temp_height = 13852*****
*****Processing temp_height = 16885*****
*****Processing temp_height = 19901*****
*****Processing temp_height = 22837*****
*****Processing temp_height = 25878*****
*****Processing temp_height = 28939*****
*****Processing temp_height = 31973*****
*****Processing temp_height = 34709*****
*****Processing temp_height = 37466*****
*****Processing temp_height = 40223*****
*****Processing temp_height = 42970*****
*****Processing temp_height = 45551*****
*****Processing temp_height = 46016*****
done
********************Processing https://careers.fenbushicapital.vc/jobs?skills=User+Experience+Design for label Design********************


/tmp/ipykernel_3487/375155834.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 4397*****
*****Processing temp_height = 6789*****
done
********************Processing https://careers.fenbushicapital.vc/jobs?markets=Stablecoin for label Operations********************


/tmp/ipykernel_3487/375155834.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 4892*****
*****Processing temp_height = 8077*****
*****Processing temp_height = 11362*****
*****Processing temp_height = 14721*****
*****Processing temp_height = 17965*****
*****Processing temp_height = 21289*****
*****Processing temp_height = 24641*****
*****Processing temp_height = 27878*****
*****Processing temp_height = 31209*****
*****Processing temp_height = 34508*****
*****Processing temp_height = 37839*****
*****Processing temp_height = 41128*****
*****Processing temp_height = 44477*****
*****Processing temp_height = 47686*****
*****Processing temp_height = 50947*****
*****Processing temp_height = 54302*****
*****Processing temp_height = 57591*****
*****Processing temp_height = 60867*****
*****Processing temp_height = 64110*****
*****Processing temp_height = 67375*****
*****Processing temp_height = 70678*****
*****Processing temp_height = 73953*****
*****Processing temp_height = 77273*****
*****Processing temp_height = 80598*****
*****Processing te

In [27]:
# show all the results
count = 0
total_columns = ['Job title', 'Type of position', 'Job Location', 'Job description', 'Working type', 'Contact', 'Achievements', 'Base salary', 'Equity']
total_values = [[] for _ in total_columns]

for single_job in job_title_list:
    count += 1
    print("*****Processing job {job}*****".format(job=single_job))
    print("Type of position: " + type_list[single_job])
    print("Job location: " + location_list[single_job])
    print("Job description: " + description_list[single_job])
    print("Working type: " + (",".join(str(x) for x in industry_list[single_job])))
    print("Contract: " + contract_list[single_job])
    print("Achievements: " + achievements_list[single_job])
    print("Base salary: " + salary_list[single_job])
    print("Equity: " + equity_list[single_job])
    
    print("label: " + (",".join(str(x) for x in label_list[single_job])))
    
    single_values = [
        single_job,
        type_list[single_job],
        location_list[single_job],
        description_list[single_job],
        ",".join(str(x) for x in industry_list[single_job]),
        contract_list[single_job],
        achievements_list[single_job],
        salary_list[single_job],
        equity_list[single_job]
    ]
    for value_index in range(len(total_columns)):
        total_values[value_index].append(single_values[value_index])
final_result = {}
for column_index in range(len(total_columns)):
    final_result.update({
        total_columns[column_index]: total_values[column_index]
    })
final_result = PD.DataFrame(final_result)

print("*****Total count {count}*****".format(count=count))
file_name = "jobs.csv"
print("*****Save the result to file {file}*****".format(file=file_name))
final_result.to_csv(file_name)


*****Processing job Finance Analyst*****
Type of position: Full-time
Job location: Bengaluru, IN
Job description: https://in.linkedin.com/jobs/view/finance-analyst-at-falconx-3425496165
Working type: Non-Tech
Contract: talentre
Achievements: Null
Base salary: Not Posted
Equity: Not Posted
label: Finance,ERP,Month End Close,NetSuite,P2P,Contract Management,Expense Reports,Procure to pay,Budgeting,Financial Statements
*****Processing job Head of Marketing - Flow Blockchain*****
Type of position: Full-time
Job location: US / Canada, Remote
Job description: https://jobs.lever.co/axiomzen/fe2c77bf-560b-4f48-a08e-b845e6a983c9?lever-origin=applied&lever-source%5B%5D=fenbushi%2520capital
Working type: Marketing
Contract: talentre
Achievements: Null
Base salary: Not Posted
Equity: Not Posted
label: Blockchain,Onboarding,Marketing
*****Processing job Business Development Director, APAC*****
Type of position: Full-time
Job location: Singapore
Job description: 
Circle is a global financial technol

# 采用静态捕获的方法(已优化)

In [ ]:
# make the bs soup into a list
with open('jobs.html', 'r', encoding='utf-8') as f:
 	soup_list = BeautifulSoup(f.read(), 'lxml').body.find('div', id="root-react-container").find("div", class_ = "boards-body fenbushi-capital").find("div", class_ = "job-list")

In [ ]:
# get the initializd infomation
tag_list = []

job_title_list = []
type_list = {}
industry_list = {}
contract_list = {}
location_list = {}
equity_list = {}
achievements_list = {}
# loop = 0

for tag in soup_list.find_all('div',class_="job-list-job"):
        # print(tag)
        job_title_list.append(tag.find('a', target = "_blank").text)
        type_list[tag.find('a', target = "_blank").text] = 'Full-time'
        industry_list[tag.find('a', target = "_blank").text] = ['Others']
        contract_list[tag.find('a', target = "_blank").text] = 'talentre'
        # location_list[tag.find('a', target = "_blank").text] = tag.find('span', class_ = "job-list-badge-text").text if (not tag.find('span', class_ = "job-list-badge-text").text[0:6] == "Posted") else "Not Posted"
        location_list[tag.find('a', target = "_blank").text] = tag.find('span', class_ = "job-list-company-meta-item").text
        equity_list[tag.find('a', target = "_blank").text] = "Not Posted"
        achievements_list
        # loop += 1
        # if(loop == 10):
        #     break
        # for tag_ in tag.find_all('span',attrs={ 'data-toggle': 'tooltip'}):
        #     if('$' in tag_.text):
        #         tmp = tag_.text.split('$')[1].replace(',','').replace(')','')


In [ ]:
# adjust the type_list
with open('jobs_internal.html', 'r', encoding='utf-8') as f:
 	soup_list_internal = BeautifulSoup(f.read(), 'lxml').body.find('div', id="root-react-container").find("div", class_ = "boards-body fenbushi-capital").find("div", class_ = "job-list")

In [ ]:
for tag in soup_list_internal.find_all('div',class_="job-list-job"):
    type_list[tag.find('a', target = "_blank").text] = 'Internship'

In [ ]:
# adjust the industry_list
industry = ['Engineer', 'Marketing']

In [ ]:
# show all the results
count = 0
df = PD.DataFrame(columns=['Job title', 'Type of position', 'Job Location', 'Job description', 'Working type', 'Contact', 'Achievements', 'Base salary', 'Equity'])
for single_job in job_title_list:
    count += 1
    print("*****Processing job {job}*****".format(job=single_job))
    print("Type of position: " + type_list[single_job])
    print("Job location: " + location_list[single_job])
    print("Working type: " + (",".join(str(x) for x in industry_list[single_job])))
    print("Contract: " + contract_list[single_job])
    print("Achievements: " + achievements_list[single_job])
    print("Equity: " + equity_list[single_job])
    print("Achievements: " + achievements_list[single_job])
print("*****Total count {count}*****".format(count=count))

*****Processing job Business Operations Intern*****
Type of position: Internship
Job location: New York City Metropolitan Area, US
Working type: Others
Contract: talentre
Equity: Not Posted
*****Processing job Digital Marketing Assistant (Intern)*****
Type of position: Internship
Job location: New York, NY, US
Working type: Others
Contract: talentre
Equity: Not Posted
*****Processing job Marketing Assistant (Intern)*****
Type of position: Internship
Job location: New York, NY, US
Working type: Others
Contract: talentre
Equity: Not Posted
*****Processing job Platform Engineering Intern*****
Type of position: Internship
Job location: New York, NY  / Seattle, WA / San Francisco Bay Area, CA / Remote
Working type: Others
Contract: talentre
Equity: Not Posted
*****Processing job Internship Spring/Summer 2023 - Software Engineer, Tools*****
Type of position: Internship
Job location: New York, NY  / Seattle, WA / San Francisco Bay Area, CA / Remote
Working type: Others
Contract: talentre
Equi